In [2]:
import code_variables
import boto3
import sagemaker
from sentence_transformers import SentenceTransformer
import pandas as pd
import string
import time
from tqdm import tqdm
import os

In [3]:
role=sagemaker.get_execution_role()
sagemaker_session=sagemaker.session.Session()
sagemaker_session.download_data('/root/Non_pipeline_implementation',code_variables.bucket_name,code_variables.file_name)

In [4]:
model=SentenceTransformer('sentence-transformers/paraphrase-TinyBERT-L6-v2')

In [5]:
dataset=pd.read_csv('Catalog2.csv',sep='|')

In [6]:
dataset.head()

,material_description,purchase_requisition_number,line_type,commodity_code,cost_center,purchase_order_number,currency,vendor_number,purchasing_organization_description,supplier_name_l1,item_number_of_purchase_requisition,requisition_date,supplier_part_number
0,ASUS valve 4GB RAM,PR5762,Catalog Item,Not Set,0000001000,Not Set,USD,[Unspecified],Not Set,[Unspecified],1,25 Feb 2018,Not Set
1,Service from Contract_2_SS,PR14117,Catalog Item,Not Set,ZCCH10,Not Set,USD,3200000040,Not Set,Mediacom,2,09 Mar 2020,Not Set
2,Service from Contract_2_SS,PR14118,Catalog Item,Not Set,ZCCH10,Not Set,USD,3200000040,Not Set,Mediacom,2,09 Mar 2020,Not Set
3,Service from Contract_2_SS,PR14567,Catalog Item,Not Set,ZCCH10,Not Set,USD,3200000040,Not Set,Mediacom,2,27 Mar 2020,Not Set
4,Pentium Processor,PR17493,Catalog Item,Not Set,Not Set,Not Set,USD,3200000040,Not Set,Mediacom,1,11 Jan 2021,Not Set


In [7]:
dataset.columns

Index(['material_description', 'purchase_requisition_number', 'line_type',
       'commodity_code', 'cost_center', 'purchase_order_number', 'currency',
       'vendor_number', 'purchasing_organization_description',
       'supplier_name_l1', 'item_number_of_purchase_requisition',
       'requisition_date', 'supplier_part_number'],
      dtype='object')

In [8]:
dataset.shape

(28767, 13)

In [9]:
dataset=dataset.drop_duplicates(subset=['material_description'], keep='last').dropna().reset_index(drop = True)

In [10]:
dataset.shape

(21112, 13)

In [11]:
embeds=[]
for desc in tqdm(dataset['material_description']):
    embeds.append(model.encode(desc))

100%|██████████| 21112/21112 [26:28<00:00, 13.29it/s]


In [12]:
dataset['embeddings']=embeds

In [14]:
dataset.to_csv(code_variables.embed_file,index=False)

In [15]:
sagemaker_session.upload_data(code_variables.embed_file,key_prefix='data',bucket=code_variables.bucket_name)

's3://aiops.d-aiml-315952988300-us-east-2/data/catalog_embeds.csv'